In [95]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

In [96]:
clinical_data = pd.read_csv('data\\NSCLCR01Radiogenomic_DATA_LABELS_2018-05-22_1500-shifted.csv', index_col=False)

In [97]:
# clinical_data = clinical_data.drop(['Case ID','Patient affiliation','%GG', 'Survival Status','Date of Last Known Alive', 'Date of Last Known Alive', 'CT Date', 'PET Date', 'Date of Recurrence'], axis=1)
clinical_data.head(2)

,Age at Histological Diagnosis,Weight (lbs),Gender,Ethnicity,Smoking status,Pack Years,Quit Smoking Year,Tumor Location (choice=RUL),Tumor Location (choice=RML),Tumor Location (choice=RLL),...,"Pleural invasion (elastic, visceral, or parietal)",EGFR mutation status,KRAS mutation status,ALK translocation status,Adjuvant Treatment,Chemotherapy,Radiation,Recurrence Location,Days between CT and surgery,Recurrence
0,34,Not Collected,Male,Not Recorded In Database,Nonsmoker,NaN,NaN,Unchecked,Unchecked,Checked,...,Not collected,Wildtype,Mutant,Wildtype,No,No,No,distant,9,yes
1,33,Not Collected,Female,Not Recorded In Database,Nonsmoker,NaN,NaN,Unchecked,Unchecked,Unchecked,...,Not collected,Wildtype,Wildtype,Not collected,No,No,No,NaN,3,no


In [98]:
clinical_data['Quit Smoking Year'] = pd.to_datetime(clinical_data['Quit Smoking Year'], format='%Y')

In [99]:
clinical_data['Quit Smoking Year']

0            NaT
1            NaT
2            NaT
3            NaT
4     1962-01-01
         ...    
206   1994-01-01
207   1993-01-01
208          NaT
209   1966-01-01
210          NaT
Name: Quit Smoking Year, Length: 211, dtype: datetime64[ns]

In [100]:
#replace Not Collected with NaN
clinical_data.replace('Not Collected', 'NaN', inplace=True)
clinical_data.replace('Not collected', 'NaN', inplace=True)
clinical_data.replace('Not Recorded In Database', 'NaN', inplace=True)
clinical_data.replace('NaN', np.nan, inplace=True)

In [101]:
#weight to numerical value
clinical_data['Weight (lbs)'] = pd.to_numeric(clinical_data['Weight (lbs)'])
# clinical_data.astype({'Weight (lbs)':'Int32'})

In [102]:
clinical_data['Quit Smoking'] = clinical_data['Quit Smoking Year'].isnull()
clinical_data['Quit Smoking Year'] = clinical_data.drop('Quit Smoking Year', axis=1)

In [103]:
clinical_data['Weight (lbs)'].fillna(-999, inplace=True)

In [104]:
dm = pd.DataFrame(pd.get_dummies(clinical_data))

In [105]:
dm = dm.drop(['Recurrence_no'], axis=1)

In [106]:
clinical_data['Weight (lbs)']

0     -999.0
1     -999.0
2     -999.0
3     -999.0
4     -999.0
       ...  
206    184.0
207    231.5
208   -999.0
209    158.0
210    229.0
Name: Weight (lbs), Length: 211, dtype: float64

In [107]:
clinical_data['Quit Smoking']

0       True
1       True
2       True
3       True
4      False
       ...  
206    False
207    False
208     True
209    False
210     True
Name: Quit Smoking, Length: 211, dtype: bool

In [108]:
#define x and y
X = dm.iloc[:, dm.columns != 'Recurrence_yes']
y = dm.iloc[:,-1].values

In [109]:
#clinical_data.iloc[:,-1]

Train Test Split

In [110]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [111]:
X_train

,Age at Histological Diagnosis,Weight (lbs),Days between CT and surgery,Quit Smoking,Gender_Female,Gender_Male,Ethnicity_African-American,Ethnicity_Asian,Ethnicity_Caucasian,Ethnicity_Hispanic/Latino,...,ALK translocation status_Wildtype,Adjuvant Treatment_No,Adjuvant Treatment_Yes,Chemotherapy_No,Chemotherapy_Yes,Radiation_No,Radiation_Yes,Recurrence Location_distant,Recurrence Location_local,Recurrence Location_regional
110,72,200.655,38,False,0,1,0,0,1,0,...,1,1,0,1,0,1,0,0,0,0
130,69,200.655,9,True,0,1,0,0,1,0,...,1,0,1,0,1,1,0,0,0,0
24,77,-999.000,13,False,1,0,0,0,0,0,...,1,1,0,1,0,1,0,0,0,0
30,66,-999.000,35,True,0,1,0,0,0,0,...,1,1,0,1,0,1,0,0,0,0
60,70,125.000,90,True,1,0,0,0,1,0,...,0,0,1,0,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,71,205.000,42,False,0,1,0,0,1,0,...,1,1,0,1,0,1,0,0,0,0
192,80,157.000,23,True,0,1,0,1,0,0,...,1,1,0,1,0,1,0,0,0,0
117,70,171.990,56,False,0,1,0,0,1,0,...,1,1,0,1,0,1,0,0,0,0
47,72,-999.000,6,False,0,1,0,0,0,0,...,1,1,0,1,0,1,0,0,0,0


Feature Scaling

In [112]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [113]:
y_train

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=uint8)

In [114]:
from sklearn.naive_bayes import GaussianNB
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

nb_y_predict = nb_classifier.predict(X_test)
nb_y_predict

from sklearn.metrics import confusion_matrix,accuracy_score,recall_score, precision_score
cm = confusion_matrix(y_test, nb_y_predict)
ac = accuracy_score(y_test,nb_y_predict)
rs = recall_score(y_test,nb_y_predict)
ps = precision_score(y_test,nb_y_predict)

print(cm)
print(ac)
print(rs)
print(ps)

[[12 34]
 [ 7 11]]
0.359375
0.6111111111111112
0.24444444444444444


In [115]:
from sklearn.svm import LinearSVC
svm_classifier = LinearSVC()
svm_classifier.fit(X_train, y_train)

svm_y_predict = svm_classifier.predict(X_test)
svm_y_predict

from sklearn.metrics import confusion_matrix,accuracy_score,recall_score, precision_score
cm = confusion_matrix(y_test, svm_y_predict)
ac = accuracy_score(y_test,svm_y_predict)
rs = recall_score(y_test,svm_y_predict)
ps = precision_score(y_test,svm_y_predict)

print(cm)
print(ac)
print(rs)
print(ps)

[[46  0]
 [ 0 18]]
1.0
1.0
1.0


In [116]:
from sklearn.tree import DecisionTreeClassifier
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)

dt_y_predict = dt_classifier.predict(X_test)
dt_y_predict

from sklearn.metrics import confusion_matrix,accuracy_score,recall_score, precision_score
cm = confusion_matrix(y_test, dt_y_predict)
ac = accuracy_score(y_test,dt_y_predict)
rs = recall_score(y_test,dt_y_predict)
ps = precision_score(y_test,dt_y_predict)

print(cm)
print(ac)
print(rs)
print(ps)

[[46  0]
 [ 0 18]]
1.0
1.0
1.0


In [117]:
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
gb_classifier = GradientBoostingClassifier()
gb_classifier.fit(X_train, y_train)

gb_y_predict = gb_classifier.predict(X_test)
gb_y_predict

from sklearn.metrics import confusion_matrix,accuracy_score,recall_score, precision_score
cm = confusion_matrix(y_test, gb_y_predict)
ac = accuracy_score(y_test,gb_y_predict)
rs = recall_score(y_test,gb_y_predict)
ps = precision_score(y_test,gb_y_predict)

print(cm)
print(ac)
print(rs)
print(ps)

[[46  0]
 [ 0 18]]
1.0
1.0
1.0


k-fold validation

In [118]:
from sklearn.model_selection import cross_val_score

In [119]:
cross_val_score(GaussianNB(), X, y)

array([0.41860465, 0.54761905, 0.69047619, 0.4047619 , 0.52380952])

In [120]:
cross_val_score(SVC(), X, y)

array([0.74418605, 0.73809524, 0.73809524, 0.73809524, 0.76190476])

In [121]:
cross_val_score(DecisionTreeClassifier(), X, y)

array([1., 1., 1., 1., 1.])

In [122]:
cross_val_score(GradientBoostingClassifier(), X, y)

array([1., 1., 1., 1., 1.])

In [123]:
X

,Age at Histological Diagnosis,Weight (lbs),Days between CT and surgery,Quit Smoking,Gender_Female,Gender_Male,Ethnicity_African-American,Ethnicity_Asian,Ethnicity_Caucasian,Ethnicity_Hispanic/Latino,...,ALK translocation status_Wildtype,Adjuvant Treatment_No,Adjuvant Treatment_Yes,Chemotherapy_No,Chemotherapy_Yes,Radiation_No,Radiation_Yes,Recurrence Location_distant,Recurrence Location_local,Recurrence Location_regional
0,34,-999.0,9,True,0,1,0,0,0,0,...,1,1,0,1,0,1,0,1,0,0
1,33,-999.0,3,True,1,0,0,0,0,0,...,0,1,0,1,0,1,0,0,0,0
2,69,-999.0,28,True,1,0,0,0,0,0,...,1,1,0,1,0,1,0,0,0,0
3,80,-999.0,47,True,1,0,0,0,0,0,...,1,1,0,1,0,1,0,0,0,0
4,76,-999.0,2,False,0,1,0,0,0,0,...,1,1,0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,75,184.0,14,False,0,1,0,0,1,0,...,0,1,0,1,0,1,0,0,0,0
207,61,231.5,72,False,0,1,0,0,1,0,...,0,0,1,0,1,1,0,0,0,0
208,52,-999.0,8,True,1,0,0,0,1,0,...,1,1,0,1,0,1,0,0,0,0
209,67,158.0,65,False,0,1,0,1,0,0,...,1,1,0,1,0,1,0,0,0,0
